# Machine Learning Model Architecture

## Construction of Test/Train datasets

In [1]:
import pandas as pd
# Let's make sure these directories are clean before we start
import shutil
try:
    shutil.rmtree("../data/project3/data_all_modified/data_split/train")
    shutil.rmtree("../data/project3/data_all_modified/data_split/test")
except:
    pass

/tmp/ipykernel_33/3444650230.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# We have two classes which contains all the data: Damage & No_damage
# Let's create directories for each class in the train and test directories.
import os
# ensure directories exist
from pathlib import Path

Path("../data/project3/data_all_modified/data_split/test/damage").mkdir(parents=True, exist_ok=True)
Path("../data/project3/data_all_modified/data_split/test/no_damage").mkdir(parents=True, exist_ok=True)

Path("../data/project3/data_all_modified/data_split/train/damage").mkdir(parents=True, exist_ok=True)
Path("../data/project3/data_all_modified/data_split/train/no_damage").mkdir(parents=True, exist_ok=True)

In [3]:
# we need paths of images for individual classes so we can copy them in the new directories that we created above
all_damage_file_paths = os.listdir('../data/project3/data_all_modified/damage')
all_no_damage_file_paths = os.listdir('../data/project3/data_all_modified/no_damage')

In [4]:
import random

train_damage_paths = random.sample(all_damage_file_paths, int(len(all_damage_file_paths)*0.8))
print("train Damage image count: ", len(train_damage_paths))
test_damage_paths = [ p for p in all_damage_file_paths if p not in train_damage_paths]
print("test Damage image count: ", len(test_damage_paths))
# ensure no overlap:
overlap = [p for p in train_damage_paths if p in test_damage_paths]
print("len of overlap: ", len(overlap))

train_no_damage_paths = random.sample(all_no_damage_file_paths, int(len(all_no_damage_file_paths)*0.8))
print("train No Damage image count: ", len(train_no_damage_paths))
test_no_damage_paths = [ p for p in all_no_damage_file_paths if p not in train_no_damage_paths]
print("test No Damage image count: ", len(test_no_damage_paths))
# ensure no overlap:
overlap = [p for p in train_no_damage_paths if p in test_no_damage_paths]
print("len of overlap: ", len(overlap))


train Damage image count:  11336
test Damage image count:  2834
len of overlap:  0
train No Damage image count:  5721
test No Damage image count:  1431
len of overlap:  0


In [5]:
#ensure to copy the images to the directories
import shutil
for p in train_damage_paths:
    shutil.copyfile(os.path.join('../data/project3/data_all_modified/damage', p), os.path.join("../data/project3/data_all_modified/data_split/train/damage", p) )

for p in test_damage_paths:
    shutil.copyfile(os.path.join('../data/project3/data_all_modified/damage', p), os.path.join("../data/project3/data_all_modified/data_split/test/damage", p) )

for p in train_no_damage_paths:
    shutil.copyfile(os.path.join('../data/project3/data_all_modified/no_damage', p), os.path.join("../data/project3/data_all_modified/data_split/train/no_damage", p) )

for p in test_no_damage_paths:
    shutil.copyfile(os.path.join('../data/project3/data_all_modified/no_damage', p), os.path.join("../data/project3/data_all_modified/data_split/test/no_damage", p) )

# check counts:
print("Files in train/damage: ", len(os.listdir("../data/project3/data_all_modified/data_split/train/damage")))
print("Files in train/no_damage: ", len(os.listdir("../data/project3/data_all_modified/data_split/test/damage")))

print("Files in test/damage: ", len(os.listdir("../data/project3/data_all_modified/data_split/train/no_damage")))
print("Files in test/no_damage: ", len(os.listdir("../data/project3/data_all_modified/data_split/test/no_damage")))

Files in train/damage:  11336
Files in train/no_damage:  2834
Files in test/damage:  5721
Files in test/no_damage:  1431


## Data Pre-Processing

In [6]:
# Using PIL to get image dimensions
from PIL import Image

def get_image_dimensions(image_path: str):
    '''
    Gets image dimensions through the input of a directory

    Input: Takes string input of the directory path to an image

    Output: Returns image dimensions of height & width pixels
    '''
    try:
        with Image.open(image_path) as img:
            width, height = img.size
            return width, height
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

def check_images_same_size(directory: str):
    '''
    Ensures that all images within a directory are the same size

    Input: Takes a string input of the desired directory to be check

    Output: Prints a statement stating images are the same size & their dimensions,
            or prints that the images have different sizes, or that there werent any valid images in
            the directory.
    '''
    dimensions_set = set()  # To store unique dimensions of images
    for filename in os.listdir(directory):
        if filename.endswith(".jpeg"):
            image_path = os.path.join(directory, filename)
            dimensions = get_image_dimensions(image_path)
            if dimensions:
                dimensions_set.add(dimensions)

    if len(dimensions_set) == 1:
        dimensions = dimensions_set.pop()
        print("All images are the same size.")
        print(f"Image dimensions: {dimensions[0]}x{dimensions[1]} pixels")
    elif len(dimensions_set) > 1:
        print("Images have different sizes.")
    else:
        print("No valid images found in the directory.")

In [7]:
train_data_dmg = '../data/project3/data_all_modified/data_split/train/damage'
train_data_no_dmg = '../data/project3/data_all_modified/data_split/train/no_damage'
check_images_same_size(train_data_dmg)
check_images_same_size(train_data_no_dmg)

All images are the same size.
Image dimensions: 128x128 pixels
All images are the same size.
Image dimensions: 128x128 pixels


In [8]:
pip install tensorflow_datasets --user

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 49.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 219.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 kB 222.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.1/156.1 kB 301.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 258.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 219.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 298.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 289.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.1/229.1 kB 311.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 311.9 MB/s eta 0:00:00
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21483 sha256=fca7643be49509ffdb34acf4765cb018dc766950e81c3fadeee301eef7

In [9]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
# Now that we know the image dimensions
train_data_dir = '../data/project3/data_all_modified/data_split/train'
# Number of images we want to process at once
batch_size = 64

# Target image size (128 px by 128 px)
img_height = 128
img_width = 128
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
train_data_dir,
validation_split=0.2,
subset="both",
seed=123,
image_size=(img_height, img_width),
batch_size=batch_size
)
rescale = Rescaling(scale=1.0/255)
train_rescale_ds = train_ds.map(lambda image,label:(rescale(image),label))
val_rescale_ds = val_ds.map(lambda image,label:(rescale(image),label))

2024-04-09 21:59:05.771940: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-09 21:59:05.806952: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-09 21:59:05.806985: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-09 21:59:05.808217: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-09 21:59:05.814566: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-09 21:59:05.815284: I tensorflow/core/platform/cpu_feature_guard.cc:1

Found 17057 files belonging to 2 classes.
Using 13646 files for training.
Using 3411 files for validation.


In [10]:
test_data_dir = '../data/project3/data_all_modified/data_split/train'

batch_size = 2

img_height = 128
img_width = 128

# note that subset="training", "validation", "both", and dictates what is returned
test_ds = tf.keras.utils.image_dataset_from_directory(
test_data_dir,
seed=123,
image_size=(img_height, img_width),
)

rescale = Rescaling(scale=1.0/255)
test_rescale_ds = test_ds.map(lambda image,label:(rescale(image),label))

Found 17057 files belonging to 2 classes.


## A Dense ANN Model

In [11]:
# Building a CNN with 3 alternating convolutional layers & pooling layers
# with 2 dense hidden layers. Output layer has 3 classes & softmax activation

from keras import layers, models, optimizers
import pandas as pd

# initialize sequential model
model_cnn = models.Sequential()

# Convolutional layer with 64 filters, and a kernel size of 3x3.
# padding = 'same' gives the same output size as input_shape
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu', padding="same", input_shape=(img_height,img_width,3)))

# Adding max pooling to reduce the size of output of first conv layer
model_cnn.add(layers.MaxPooling2D((2, 2), padding = 'same'))

# Second Convolutional layer with Pooling layer to reduce size
model_cnn.add(layers.Conv2D(32, (3, 3), activation='relu', padding="same"))
model_cnn.add(layers.MaxPooling2D((2, 2), padding = 'same'))

# Final convolutional layer & pooling to reduce size
model_cnn.add(layers.Conv2D(32, (3, 3), activation='relu', padding="same"))
model_cnn.add(layers.MaxPooling2D((2, 2), padding = 'same'))

# flattening the output of the conv layer after max pooling 
# makes it ready for creating dense connections
model_cnn.add(layers.Flatten())

# Adding a fully connected dense layer with 100 neurons
model_cnn.add(layers.Dense(100, activation='relu'))

# Adding a fully connected dense layer with 84 neurons
model_cnn.add(layers.Dense(84, activation='relu'))

# Adding the output layer with 2 neurons and 
# activation functions as softmax since this is a multi-class classification problem
model_cnn.add(layers.Dense(2, activation='softmax'))

In [12]:
# Compile model
# RMSprop (Root Mean Square Propagation) is commonly used in training deep neural networks.
model_cnn.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])

# Generating the summary of the model
model_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 64, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        18464     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 32, 32, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 16, 16, 32)        0

In [13]:
#fit the model from image generator
history = model_cnn.fit(
            train_rescale_ds,
            batch_size=64,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
214/214 [==============================] - 107s 491ms/step - loss: 0.5947 - accuracy: 0.7092 - val_loss: 0.6913 - val_accuracy: 0.6722
Epoch 2/20
214/214 [==============================] - 106s 494ms/step - loss: 0.4646 - accuracy: 0.7954 - val_loss: 0.4194 - val_accuracy: 0.7986
Epoch 3/20
214/214 [==============================] - 106s 497ms/step - loss: 0.4091 - accuracy: 0.8253 - val_loss: 0.3344 - val_accuracy: 0.8690
Epoch 4/20
214/214 [==============================] - 103s 483ms/step - loss: 0.3589 - accuracy: 0.8507 - val_loss: 0.2929 - val_accuracy: 0.8898
Epoch 5/20
214/214 [==============================] - 95s 446ms/step - loss: 0.3168 - accuracy: 0.8723 - val_loss: 0.2493 - val_accuracy: 0.9006
Epoch 6/20
214/214 [==============================] - 95s 443ms/step - loss: 0.2782 - accuracy: 0.8890 - val_loss: 0.2520 - val_accuracy: 0.9050
Epoch 7/20
214/214 [==============================] - 99s 462ms/step - loss: 0.2357 - accuracy: 0.9089 - val_loss: 0.2595 - va

In [14]:
test_loss, test_accuracy = model_cnn.evaluate(test_rescale_ds, verbose = 0)
# validation accuracy
print(f'Test Loss: {test_loss}')
# test accuracy
print(f'Test Accuracy: {test_accuracy}')

Test Loss: 0.10109329223632812
Test Accuracy: 0.959254264831543


## LeNet-5 Architecture

In [15]:
model_lenet5 = models.Sequential()

# Layer 1: Convolutional layer with 6 filters of size 3x3, followed by average pooling
model_lenet5.add(layers.Conv2D(6, kernel_size=(3, 3), activation='relu', input_shape=(img_height,img_width,3)))
model_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Layer 2: Convolutional layer with 16 filters of size 3x3, followed by average pooling
model_lenet5.add(layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))
model_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Flatten the feature maps to feed into fully connected layers
model_lenet5.add(layers.Flatten())

# Layer 3: Fully connected layer with 120 neurons
model_lenet5.add(layers.Dense(120, activation='relu'))

# Layer 4: Fully connected layer with 84 neurons
model_lenet5.add(layers.Dense(84, activation='relu'))

# Output layer: Fully connected layer with num_classes neurons (e.g., 2 )
model_lenet5.add(layers.Dense(2, activation='softmax'))

# Compile model
model_lenet5.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), 
                     loss='sparse_categorical_crossentropy', 
                     metrics=['accuracy'])

# Generating the summary of the model
model_lenet5.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 126, 126, 6)       168       
                                                                 
 average_pooling2d (Average  (None, 63, 63, 6)         0         
 Pooling2D)                                                      
                                                                 
 conv2d_4 (Conv2D)           (None, 61, 61, 16)        880       
                                                                 
 average_pooling2d_1 (Avera  (None, 30, 30, 16)        0         
 gePooling2D)                                                    
                                                                 
 flatten_1 (Flatten)         (None, 14400)             0         
                                                                 
 dense_3 (Dense)             (None, 120)              

In [16]:
history = model_lenet5.fit(
            train_rescale_ds,
            batch_size=64,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
214/214 [==============================] - 19s 84ms/step - loss: 0.6060 - accuracy: 0.6908 - val_loss: 0.5792 - val_accuracy: 0.7227
Epoch 2/20
214/214 [==============================] - 18s 84ms/step - loss: 0.4675 - accuracy: 0.8057 - val_loss: 0.4176 - val_accuracy: 0.8050
Epoch 3/20
214/214 [==============================] - 18s 82ms/step - loss: 0.3774 - accuracy: 0.8576 - val_loss: 0.4992 - val_accuracy: 0.7710
Epoch 4/20
214/214 [==============================] - 18s 82ms/step - loss: 0.3449 - accuracy: 0.8717 - val_loss: 0.3520 - val_accuracy: 0.8593
Epoch 5/20
214/214 [==============================] - 18s 85ms/step - loss: 0.3253 - accuracy: 0.8815 - val_loss: 0.3046 - val_accuracy: 0.8930
Epoch 6/20
214/214 [==============================] - 19s 89ms/step - loss: 0.3087 - accuracy: 0.8881 - val_loss: 0.3153 - val_accuracy: 0.8807
Epoch 7/20
214/214 [==============================] - 18s 82ms/step - loss: 0.2878 - accuracy: 0.8950 - val_loss: 0.2889 - val_accuracy:

In [17]:
test_loss, test_accuracy = model_lenet5.evaluate(test_rescale_ds, verbose = 0)
# validation accuracy
print(f'Test Loss: {test_loss}')
# test accuracy
print(f'Test Accuracy: {test_accuracy}')

Test Loss: 0.16592474281787872
Test Accuracy: 0.9392038583755493


## Modified LeNet-5

In [18]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
# Now that we know the image dimensions
train_data_dir = 'data/project3/data_all_modified/data_split/train'
# Number of images we want to process at once
batch_size = 64

# Target image size (128 px by 128 px)
img_height = 128
img_width = 128
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
train_data_dir,
validation_split=0.2,
subset="both",
seed=123,
image_size=(img_height, img_width),
batch_size=batch_size
)
rescale = Rescaling(scale=1.0/255)
train_rescale_ds = train_ds.map(lambda image,label:(rescale(image),label))
val_rescale_ds = val_ds.map(lambda image,label:(rescale(image),label))

Found 17057 files belonging to 2 classes.
Using 13646 files for training.
Using 3411 files for validation.


In [19]:
test_data_dir = 'data/project3/data_all_modified/data_split/train'

batch_size = 2

img_height = 128
img_width = 128

# note that subset="training", "validation", "both", and dictates what is returned
test_ds = tf.keras.utils.image_dataset_from_directory(
test_data_dir,
seed=123,
image_size=(img_height, img_width),
)

rescale = Rescaling(scale=1.0/255)
test_rescale_ds = test_ds.map(lambda image,label:(rescale(image),label))

Found 17057 files belonging to 2 classes.


In [20]:
from keras import layers
from keras import models
import pandas as pd

model_lenet5 = models.Sequential()
# Layer 1: Convolutional layer with 6 filters of size 3x3, followed by Max pooling
model_lenet5.add(layers.Conv2D(6, kernel_size=(3, 3), activation='relu', input_shape=(img_height,img_width,3)))
model_lenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))
# Layer 2: Convolutional layer with 32 filters of size 3x3, followed by Max pooling
model_lenet5.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(img_height,img_width,3)))
model_lenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))
# Layer 3: Convolutional layer with 64 filters of size 3x3, followed by Max pooling
model_lenet5.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(img_height,img_width,3)))
model_lenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))
# Layer 4: Convolutional layer with 128 filters of size 3x3, followed by Max pooling
model_lenet5.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(img_height,img_width,3)))
model_lenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))
# Layer 5: Convolutional layer with 128 filters of size 3x3, followed by Max pooling
model_lenet5.add(layers.Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=(img_height,img_width,3)))
model_lenet5.add(layers.MaxPooling2D(pool_size=(2, 2)))
# flattening the output of the conv layer after max pooling 
model_lenet5.add(layers.Flatten())
#Dropout Layer
model_lenet5.add(layers.Dropout(0.2))
# Layer 8: Fully connected layer with 512 neurons
model_lenet5.add(layers.Dense(512, activation='relu'))
# Output layer: Fully connected layer with num_classes neurons (e.g., 2 )
model_lenet5.add(layers.Dense(2, activation='softmax'))

model_lenet5.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), 
                     loss='sparse_categorical_crossentropy', 
                     metrics=['accuracy'])

# Generating the summary of the model
model_lenet5.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 126, 126, 6)       168       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 63, 63, 6)         0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 61, 61, 32)        1760      
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 30, 30, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 14, 14, 64)       

In [21]:
history = model_lenet5.fit(
            train_rescale_ds,
            batch_size=64,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
214/214 [==============================] - 29s 131ms/step - loss: 0.6243 - accuracy: 0.6716 - val_loss: 0.5944 - val_accuracy: 0.6693
Epoch 2/20
214/214 [==============================] - 30s 141ms/step - loss: 0.4743 - accuracy: 0.7796 - val_loss: 0.4676 - val_accuracy: 0.7684
Epoch 3/20
214/214 [==============================] - 33s 154ms/step - loss: 0.4116 - accuracy: 0.8259 - val_loss: 0.4123 - val_accuracy: 0.8150
Epoch 4/20
214/214 [==============================] - 32s 147ms/step - loss: 0.3730 - accuracy: 0.8484 - val_loss: 0.3524 - val_accuracy: 0.8575
Epoch 5/20
214/214 [==============================] - 31s 144ms/step - loss: 0.3334 - accuracy: 0.8642 - val_loss: 0.3662 - val_accuracy: 0.8467
Epoch 6/20
214/214 [==============================] - 31s 147ms/step - loss: 0.2979 - accuracy: 0.8816 - val_loss: 0.2418 - val_accuracy: 0.9024
Epoch 7/20
214/214 [==============================] - 32s 151ms/step - loss: 0.2635 - accuracy: 0.8953 - val_loss: 0.2324 - val_ac

In [21]:
test_loss, test_accuracy = model_lenet5.evaluate(test_rescale_ds, verbose = 0)
# validation accuracy
print(f'Test Loss: {test_loss}')
# test accuracy
print(f'Test Accuracy: {test_accuracy}')


Test Loss: 0.116956427693367
Test Accuracy: 0.9566991925239563
